In [1]:
from keras.preprocessing import image
from keras.models import Model, model_from_json
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten, AveragePooling2D
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
import numpy as np
import pandas as pd
import cv2
import math
import scipy.io as scio
import os
import matplotlib.pyplot as plt
%matplotlib inline
from utils import generate_generator, gen_img_paths_and_labels, loss_APE, LossHistory, SaveModelOnAPE
from sklearn.utils import shuffle


/home/chengxg/anaconda2/envs/zp_py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [8]:
# Generate the random indices for validation
split_idx = (np.asarray([0, 2400, 5160, 7920, 10080, 11880, 13440])*5).tolist()
idx_validate = 1
for idx_validate in range(1, 7):
    print(idx_validate)
    valid_idx_for_test = list(range(split_idx[idx_validate - 1], split_idx[idx_validate]))
    num_5_sec = 0
    indices_test = []
    for idx_del_0_5 in range(len(valid_idx_for_test)-1, -1, -1):
        if idx_del_0_5 % 5 == 0:
            valid_idx_for_test.pop(idx_del_0_5)
            num_5_sec += 1
            indices_test.append(idx_del_0_5)
    indices_test = sorted(indices_test, reverse=True)
    # Now the list has removed the elements whose indices are *0s or *5s
    # Then I need a for loop to randomly select one element from each 2 minutes.
    print(num_5_sec, len(valid_idx_for_test))
    indices_rand = []
    for i in range(0, num_5_sec*4, 4*12*2):
        indices_rand.append(valid_idx_for_test[i+0: i+4*12*2][np.random.randint(0, 4*12*2)] - split_idx[idx_validate])
    print('Saving...')
    np.savetxt('./indices_test_or_5_val_{}.txt'.format(idx_validate), indices_test)
    np.savetxt('./indices_rand_val_no_5_val_{}.txt'.format(idx_validate), indices_rand)

1
2400 9600
Saving...
2
2760 11040
Saving...
3
2760 11040
Saving...
4
2160 8640
Saving...
5
1800 7200
Saving...
6
1560 6240
Saving...


In [9]:
# Data import
split_idx = (np.asarray([0, 2400, 5160, 7920, 10080, 11880, 13440])*5).tolist()
idx_validate = 2
data = pd.read_excel('./imgPaths_labels_isSensor_new.xlsx')
paths, labels, sensor = data['img_path'], data['label'], data['sensor']
paths = np.squeeze(np.asarray(paths)).tolist()
labels = np.squeeze(np.asarray(labels)).tolist()
sensor = np.squeeze(np.asarray(sensor)).tolist()
train_paths = paths[:split_idx[idx_validate-1]] + paths[split_idx[idx_validate]:]
test_paths = paths[split_idx[idx_validate-1]:split_idx[idx_validate]]
train_labels = labels[:split_idx[idx_validate-1]] + labels[split_idx[idx_validate]:]
test_labels = labels[split_idx[idx_validate-1]:split_idx[idx_validate]]
train_sensor = sensor[:split_idx[idx_validate-1]] + sensor[split_idx[idx_validate]:]
test_sensor = sensor[split_idx[idx_validate-1]:split_idx[idx_validate]]

test_paths = np.asarray(test_paths).reshape(len(test_paths), 1)
test_labels= np.asarray(test_labels).reshape(len(test_labels), 1)

idx_real = np.where(np.asarray(test_sensor) == 1)
test_paths_real = test_paths[idx_real]
test_labels_real = test_labels[idx_real]
idx_validate = np.loadtxt('./indices_rand_val_no_5_val_{}.txt'.format(idx_validate)).astype(int)
validate_paths = test_paths[idx_validate]
validate_labels = test_labels[idx_validate]

test_paths = np.squeeze(test_paths).tolist()
test_labels = np.squeeze(test_labels).tolist()
test_paths_real = np.squeeze(test_paths_real).tolist()
test_labels_real = np.squeeze(test_labels_real).tolist()

validate_paths, validate_labels = np.squeeze(validate_paths).tolist(), np.squeeze(validate_labels).tolist()

print('data[:3]:\n', data[:3])
print('len(train_paths), len(train_labels):', len(train_paths), len(train_labels))
print('len(validate_paths), len(validate_labels):', len(validate_paths), len(validate_labels))
print('len(test_paths), len(test_labels):', len(test_paths), len(test_labels))

data[:3]:
    index                                          img_path      label  sensor
0      0  ./split_frames_new/1_point_05/1_point_05_0_0.jpg  72.895116       0
1      1  ./split_frames_new/1_point_05/1_point_05_0_1.jpg  72.895116       0
2      2  ./split_frames_new/1_point_05/1_point_05_0_2.jpg  72.895116       0
len(train_paths), len(train_labels): 53400 53400
len(validate_paths), len(validate_labels): 115 115
len(test_paths), len(test_labels): 13800 13800


In [10]:
# Data import
split_idx = (np.asarray([0, 2400, 5160, 7920, 10080, 11880, 13440])*5).tolist()
idx_validate = 1
data = pd.read_excel('./imgPaths_labels_isSensor_new.xlsx')
paths, labels, sensor = data['img_path'], data['label'], data['sensor']
paths = np.squeeze(np.asarray(paths)).tolist()
labels = np.squeeze(np.asarray(labels)).tolist()
sensor = np.squeeze(np.asarray(sensor)).tolist()
train_paths = paths[:split_idx[idx_validate-1]] + paths[split_idx[idx_validate]:]
test_paths = paths[split_idx[idx_validate-1]:split_idx[idx_validate]]
train_labels = labels[:split_idx[idx_validate-1]] + labels[split_idx[idx_validate]:]
test_labels = labels[split_idx[idx_validate-1]:split_idx[idx_validate]]
train_sensor = sensor[:split_idx[idx_validate-1]] + sensor[split_idx[idx_validate]:]
test_sensor = sensor[split_idx[idx_validate-1]:split_idx[idx_validate]]

test_paths = np.asarray(test_paths).reshape(len(test_paths), 1)
test_labels= np.asarray(test_labels).reshape(len(test_labels), 1)

idx_real = np.where(np.asarray(test_sensor) == 1)
test_paths_real = test_paths[idx_real]
test_labels_real = test_labels[idx_real]
idx_validate = np.loadtxt('./indices_rand_val_no_5_val_{}.txt'.format(idx_validate)).astype(int)
validate_paths = test_paths[idx_validate]
validate_labels = test_labels[idx_validate]

test_paths = np.squeeze(test_paths).tolist()
test_labels = np.squeeze(test_labels).tolist()
test_paths_real = np.squeeze(test_paths_real).tolist()
test_labels_real = np.squeeze(test_labels_real).tolist()

validate_paths, validate_labels = np.squeeze(validate_paths).tolist(), np.squeeze(validate_labels).tolist()

print('data[:3]:\n', data[:3])
print('len(train_paths), len(train_labels):', len(train_paths), len(train_labels))
print('len(validate_paths), len(validate_labels):', len(validate_paths), len(validate_labels))
print('len(test_paths), len(test_labels):', len(test_paths), len(test_labels))

data[:3]:
    index                                          img_path      label  sensor
0      0  ./split_frames_new/1_point_05/1_point_05_0_0.jpg  72.895116       0
1      1  ./split_frames_new/1_point_05/1_point_05_0_1.jpg  72.895116       0
2      2  ./split_frames_new/1_point_05/1_point_05_0_2.jpg  72.895116       0
len(train_paths), len(train_labels): 55200 55200
len(validate_paths), len(validate_labels): 100 100
len(test_paths), len(test_labels): 12000 12000


In [11]:
import random

In [15]:
from sklearn.utils import shuffle

In [16]:
shuffle(range(10))

[0, 4, 5, 2, 3, 8, 6, 1, 7, 9]